In [12]:
# Import app
from celery_basics import app
from celery_basics import add
from celery_basics import make_pi
from celery_basics import apply_function
from celery_basics import apply_counter

#### Sum 1 to 2

In [13]:
res = add.apply_async(args=(1, 2))
results = res.get()
results

IOError: Socket closed

#### Calculate pi in range 1 to 10**5

In [ ]:
%%timeit
from celery import group
job = group(make_pi.subtask((10**x, )) for x in range(0,8))
job_result = job.delay()
results = job_result.get()
print(results)

In [ ]:
%%timeit
pis = []
for x in range(0,8):
    pis.append(make_pi(10**x))
print(pis)

#### Sum the numbers between the 1 to 1000 twice

In [ ]:
%%timeit
from celery import chain
jobs = chain(add.si(i, i) for i in range(1,1001))
job_result = jobs.apply_async()
results = job_result.get()
print(results)

In [ ]:
%%timeit
results = sum([add(i, i) for i in range(1,1001)])
print(results)

#### Counter of the words in reviews summaries

In [ ]:
import pandas as pd
from collections import Counter

In [ ]:
reviews = pd.read_csv("reviews.csv", encoding="utf-8", iterator=True, chunksize=5000)
counter = Counter()

In [ ]:
for review in reviews:
    counter += review.summary.apply(lambda x :Counter(str(x).split(" "))).values.sum()

In [ ]:
counter.most_common()

#### Counter of the words in reviews summaries with Celery

In [ ]:
reviews = pd.read_csv("reviews.csv", encoding="utf-8", iterator=True, chunksize=5000)

In [ ]:
import pickle
jobs =[]
for review in reviews:
    job = apply_counter.apply_async(args=(pickle.dumps(review),))
    jobs.append(job)
counter = Counter()
for job in jobs:
    counter += pickle.loads(job.get()) 

In [ ]:
counter.sum().most_common()